In [1]:
import torch
from torch import nn

In [2]:
from d2l import torch as d2l
class AttentionDecoder(d2l.Decoder):
    def __init__(self,**kwargs):
        super(AttentionDecoder,self).__init__(**kwargs)

    @property
    def attention_weight(self):
        raise NotImplementedError

In [ ]:
class seq2seqAttentionDecoder(AttentionDecoder):
    def __init__(self,vocab_size,
                 embed_size,
                 num_hiddens,
                 num_layer,
                 droupt=0, **kwargs):
        super().__init__(**kwargs)

        self.attention = d2l.AdditiveAttention(num_hiddens,droupt)

        self.embedding = nn.Embedding(vocab_size,embed_size)

        self.rnn = nn.GRU(embed_size + num_hiddens,num_hiddens,num_layer,dropout=droupt)

        self.dense = nn.Linear(num_hiddens,vocab_size)

    def init_state(self, enc_outputs,enc_valid_lens, *args):
        outputs,hidden_state = enc_outputs

        return (outputs.permute(1,0,2),hidden_state,enc_valid_lens)

    def forward(self, X,state):
        enc_outputs , hidden_state ,enc_valid_lens = state

        X = self.embedding(X).permute(1,0,2)

        outputs,self._attention_weights = [],[]

        for x in X:
            query = torch.unsqueeze(hidden_state[-1],dim=1)
            
            context = self.attention(query,enc_outputs,enc_outputs,enc_valid_lens)

            x = torch.cat((context,torch.unsqueeze(x,dim=1)),dim=-1)

            out,hidden_state = self.rnn(x.permute(1,0,2),hidden_state)

            outputs.append(out)

            self._attention_weights.append(self.attention.attention_weights)

        outputs = self.dense(torch.cat(outputs,dim=0))
        return outputs.permute(1,0,2),[enc_outputs,hidden_state,enc_valid_lens]